In [ ]:
import glob
import pandas as pd
output_file = "../exports/filtered_patient_reports_with_event_log/combined.pkl"
batch_files = sorted(glob.glob("../exports/filtered_patient_reports_with_event_log/batch_*.pkl"))
combined_df = pd.concat([pd.read_pickle(f) for f in batch_files], ignore_index=True)
combined_df.to_pickle(output_file)

In [ ]:
output_file_embedder = "../exports/filtered_patient_reports_with_event_log_only_embedders/batch_00000000.pkl"
output_file_llm = "../exports/filtered_patient_reports_with_event_log copy/batch_00000000.pkl"
df_embedder = pd.read_pickle(output_file_embedder)
df_llm = pd.read_pickle(output_file_llm)


In [ ]:
global_list = []
def get_known_events(x):
    result = []
    for i in x:
        event = list(i.values())[0]['event']
        event_name = event[0]
        similarity = event[1]
        sentence = list(i.keys())[0]
        if similarity >= 0.2 and similarity <= 2:
            summ = {"sentence":sentence,
                           "event_name":event_name,
                           "similarity":similarity}
            result.append(summ)
            # print(summ)
            global_list.append(summ)
    return result


    
combined_df["OnlyEvents"]=combined_df["Events"].apply(get_known_events)
df_summ = pd.DataFrame(global_list)
# df_summ['sentence'] = df_summ['sentence'].apply(lambda x: x.strip())
df_summ['sentence_copy'] = df_summ['sentence'].str.strip()
df_summ['sentence_copy'] = df_summ['sentence_copy'].str.lower()
df_summ['sentence_copy'] = df_summ['sentence_copy'].str.rstrip('.')
df_summ['sentence_copy'] = df_summ['sentence_copy'].apply(lambda x: x.replace("  "," "))
df_summ['num_words'] = df_summ['sentence_copy'].apply(lambda x: len(x.split()))
df_summ = df_summ[df_summ.num_words >= 2]
df_summ.to_excel("../exports/temporary_view_0.35to0.37.xlsx")

# # combined_df.to_pickle(output_file)
# print(len(df_summ[df_summ.event_name=="Sleep"]))
# N1 = len(df_summ[df_summ.event_name=="Sleep"])
print(len(df_summ),len(df_summ[df_summ.duplicated("sentence_copy",keep="first")]))
# df_summ[df_summ.duplicated("sentence_copy",keep="")]



In [ ]:
len(df_summ[df_summ.duplicated("sentence_copy",keep=False)].sort_values("sentence_copy")["sentence_copy"].unique())

In [ ]:
sentence_count_df = df_summ["sentence_copy"].value_counts().reset_index()
print(sentence_count_df)
sentence_count_df.to_excel("../exports/frequency_of_sentences.xlsx")
repeated_sentences = sentence_count_df[sentence_count_df['count']>1]
non_repeated_sentences = sentence_count_df[sentence_count_df['count']==1]

repeated_sentences['count'].sum(), len(repeated_sentences) ,non_repeated_sentences['count'].sum()

In [ ]:
import matplotlib.pyplot as plt
plt.bar(sentence_count_df.iloc[:1000].index,sentence_count_df.iloc[:1000]['count'])


In [ ]:

def count_term(sentence,search_term=["sleep", "slept"]):
        return sum(sentence.lower().count(term) for term in search_term)

df_summ['Sleep_appears'] = df_summ['sentence'].apply(count_term)


df_summ[df_summ['Sleep_appears']>0].to_excel("../exports/temporary_view_sleep_appears.xlsx")
df_summ[df_summ['Sleep_appears']==0].to_excel("../exports/temporary_view_sleep_does_not_appear.xlsx")



len(df_summ[df_summ.Sleep_appears>0]), sum(df_summ["Sleep_appears"])
N2= len(df_summ[df_summ.Sleep_appears>0])

print(N1, N2, N2/N1*100)

In [ ]:
import pandas as pd
df = pd.read_excel("../exports/combined_event_log_dictionary_biolord.xlsx")

In [ ]:
df.columns

In [ ]:
def value_counts(list_of_dict):
    lst = [list]
    counts = {}
    for item in lst:
        counts[item] = counts.get(item, 0) + 1
    return counts
combined_df["EventCounts"] = combined_df["OnlyEvents"].apply(value_counts)
combined_df["EventCounts"]


In [ ]:

combined_df["Sleep_count"] = combined_df["EventCounts"].apply(lambda x: x.get("Sleep",0))
combined_df["Pain_count"] = combined_df["EventCounts"].apply(lambda x: x.get("Pain",0))
combined_df["Others_count"] = combined_df["EventCounts"].apply(lambda x: x.get("Others",0))

In [ ]:
combined_df.describe()


In [ ]:
from pprint import pprint
# pprint(combined_df[combined_df.Sleep_count == 40]['Events'].iloc[0])

In [ ]:
combined_df.to_csv("../exports/temporary_view.csv")

In [ ]:
len(combined_df[combined_df.Sleep_count == 40]['Events'].iloc[0])

In [ ]:
combined_df[combined_df.Sleep_count == 40]['Events'].to_excel("../exports/temporary_view.xlsx")

In [ ]:
from pprint import pprint
pprint(combined_df[combined_df.Sleep_count == 40]['Events'].iloc[0])

In [ ]:
combined_df.groupby("HADM_ID")[['Sleep_count', 'Pain_count']].sum().reset_index().describe()

In [ ]:
import pandas as pd
from pprint import pprint
import glob
import pandas as pd
pd.set_option('display.max_rows', None)

for model_type in ["dictionary", "biolord"]:
    output_folder = f"../exports/selected_reports_with_event_log_only_{model_type}"
    output_file = f"{output_folder}/combined.pkl"
    batch_files = sorted(glob.glob(f"{output_folder}/batch_*.pkl"))
    combined_df = pd.concat([pd.read_pickle(f) for f in batch_files], ignore_index=True)
    combined_df.to_pickle(output_file)

def prepare_df(df, type="biolord"):
    df = df.copy()
    df['Sent_ID'] = df['Events'].apply(lambda x: [i for i in range(len(x))])
    df = df.explode(["Sent_ID","Events"])
    df['Event_Name'] = df['Events'].apply(lambda x: x['event'])
    df['Sentence'] = df['Events'].apply(lambda x: x['sentence'])
    
    if type == "dictionary":
        df['Keyword'] = df['Events'].apply(lambda x: x['keyword'])
    if type == "biolord":
        df['Similarity'] = df['Events'].apply(lambda x: x['similarity'])
        df["Sleep_similarity"] = df['Similarity'].apply(lambda x:x["Sleep"])
        df["Pain_similarity"] = df['Similarity'].apply(lambda x:x["Pain"])
        df["Excretion_similarity"] = df['Similarity'].apply(lambda x:x["Excretion"])
        df["Eating_similarity"] = df['Similarity'].apply(lambda x:x["Eating"])
        df["Family_similarity"] = df['Similarity'].apply(lambda x:x["Family"])
        df["Alertness_similarity"] = df['Similarity'].apply(lambda x:x["Alert And Oriented"])
    return df
df_dictionary = prepare_df(pd.read_pickle("../exports/selected_reports_with_event_log_only_dictionary/combined.pkl"),type="dictionary")
df_biolord = prepare_df(pd.read_pickle("../exports/selected_reports_with_event_log_only_biolord/combined.pkl"), type = "biolord")



# pprint(df1.iloc[0].to_string())


In [ ]:
len(df_biolord), len(df_dictionary)

In [ ]:
df_dictionary.columns

In [ ]:
df_dictionary.Event_Name.value_counts()

In [ ]:
df_biolord.Sleep_similarity.hist()

In [ ]:
df_biolord[df_biolord.Event_Name=="Sleep"].Sleep_similarity.hist()

In [ ]:
12*3600/567000

In [ ]:
df_dictionary.to_excel("../exports/selected_reports_with_event_log_only_dictionary/combined.xlsx", index=False)

In [ ]:
df_dictionary.columns, df_biolord.columns
df_biolord.Similarity = df_biolord.Similarity.astype(str)
df_both = pd.merge(df_dictionary[['ROW_ID','Sent_ID','HADM_ID','CHARTTIME','STORETIME','Sentence','Event_Name','Keyword','CGID']], 
         df_biolord[['ROW_ID','Sent_ID','HADM_ID','CHARTTIME','STORETIME','Sentence','Event_Name','CGID','Eating_similarity', 'Excretion_similarity', 'Family_similarity', 'Alertness_similarity', 'Pain_similarity', 'Sleep_similarity']], 
         on=['HADM_ID','ROW_ID','Sent_ID'], how='outer',suffixes=("_dictionary","_biolord")).sort_values(by=['HADM_ID','ROW_ID','Sent_ID'])

In [ ]:
temp = df_dictionary["Event_Name_tuple"].value_counts().reset_index()
# temp
temp[temp.Event_Name_tuple != ("Unknown",)].sum(),temp[temp.Event_Name_tuple == ("Unknown",)].sum()


In [ ]:
# df_biolord.to_excel("../exports/combined_event_log_dictionary_biolord.xlsx", index=False)

In [ ]:
df_dictionary.columns

In [ ]:
df_dictionary['Event_Name_tuple'] = df_dictionary['Event_Name'].apply(tuple)
df_dictionary['Keyword_tuple'] = df_dictionary['Keyword'].apply(tuple)


In [ ]:
df_dictionary['Event_Name_tuple'] 

In [ ]:
df_dictionary

In [ ]:
df_dictionary[['Event_Name_tuple','Keyword_tuple']].value_counts().reset_index().sort_values(by=("Event_Name_tuple",'Keyword_tuple'),ascending=True)

In [ ]:
df_dictionary[['Event_Name_tuple','Keyword_tuple']].value_counts().reset_index()

In [ ]:
df_dictionary['Event_Name'].iloc[0]

In [ ]:
df_dictionary.head(10)

In [ ]:
import re
def remove_illegal_chars(val):
    if isinstance(val, str):
        # Remove characters disallowed in Excel
        return re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", "", val)
    return val

# Apply to all string cells
df_both_clean = df_both.applymap(remove_illegal_chars)
df_both.to_excel("../exports/combined_event_log_dictionary_biolord.xlsx", index=False)

In [ ]:

df_dictionary[['Event_Name','Keyword']].value_counts().reset_index().sort_values(by=['Event_Name', 'count'], ascending =[True,False])

In [ ]:
# df_dictionary[df_dictionary['Event_Name']=="Alert and Oriented"]["Sentences"]
df_dictionary[['Event_Name', 'Keyword']].value_counts()

In [ ]:
df_dictionary[(df_dictionary.Event_Name=="Others")&(df_dictionary.Keyword == "family")]	